In [169]:
import os

In [170]:
!pip install langchain-groq


In [171]:

import groq
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks.base import BaseCallbackHandler
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2


In [172]:
load_dotenv()

True

In [173]:
key=os.getenv("GROQ_API_KEY")

In [174]:
llm = ChatGroq(groq_api_key=key,model_name="llama3-8b-8192",temperature=0.7  # Directly pass the API key
)

In [175]:
with open("Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [176]:
RESPONSE_JSON

{'1': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'mcq': 'multiple choice question',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [177]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
do not need additional information like "'Here are 5 multiple choice questions for biology students based on the provided text:"
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [178]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [179]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [180]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [181]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [182]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [183]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [184]:
PATH = r"C:\Users\HP\OneDrive - ABES\Desktop\MCQ_Project\data.txt"


In [185]:
with open(PATH, 'r') as file:
    TEXT = file.read()

In [186]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [187]:
TEXT
NUMBER=5 
SUBJECT="biology"
TONE="simple"
RESPONSE_JSON=RESPONSE_JSON

In [188]:
class TokenUsageCallback(BaseCallbackHandler):
    def on_token_usage(self, tokens_used):
        print(f"Tokens used: {tokens_used}")

# Initialize callback handler
callback_handler = TokenUsageCallback()

In [189]:
def generate_evaluate_chain_with_callback(data):
    # Generate quiz and review outputs using the SequentialChain
    response = generate_evaluate_chain.invoke(data)
    
    # Example of token usage (use actual token usage data from the model if available)
    callback_handler.on_token_usage(100)  # Placeholder for token usage tracking
    
    return response

In [190]:
response = generate_evaluate_chain_with_callback(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of thei

In [191]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [192]:
quiz=response.get("quiz")

In [198]:
quiz = quiz.replace("\n","")

In [195]:
quiz

'{"1": {"mcq": "What is the scientific study of life?", "options": {"a": "Chemistry", "b": "Biology", "c": "Physics", "d": "Mathematics"}, "correct": "b"}, "2": {"mcq": "What is a major theme in biology that explains the unity and diversity of life?", "options": {"a": "Energy processing", "b": "Evolution", "c": "Cell structure", "d": "Photosynthesis"}, "correct": "b"}, "3": {"mcq": "What do biologists use to make observations, pose questions, generate hypotheses, perform experiments, and form conclusions?", "options": {"a": "The scientific method", "b": "The theory of evolution", "c": "Cell theory", "d": "The five-kingdom system"}, "correct": "a"}, "4": {"mcq": "What is the estimated age of life on Earth?", "options": {"a": "1 billion years", "b": "3.7 billion years", "c": "10 billion years", "d": "5 billion years"}, "correct": "b"}, "5": {"mcq": "What is the term used to describe the variety of different species in an ecosystem?", "options": {"a": "Biodiversity", "b": "Evolutionary ad

In [199]:
ans=json.loads(quiz)

In [202]:
type(ans)

dict

In [203]:
quiz_table_data = []
for key, value in ans.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [204]:
quiz_table_data


[{'MCQ': 'What is the scientific study of life?',
  'Choices': 'a: Chemistry | b: Biology | c: Physics | d: Mathematics',
  'Correct': 'b'},
 {'MCQ': 'What is a major theme in biology that explains the unity and diversity of life?',
  'Choices': 'a: Energy processing | b: Evolution | c: Cell structure | d: Photosynthesis',
  'Correct': 'b'},
 {'MCQ': 'What do biologists use to make observations, pose questions, generate hypotheses, perform experiments, and form conclusions?',
  'Choices': 'a: The scientific method | b: The theory of evolution | c: Cell theory | d: The five-kingdom system',
  'Correct': 'a'},
 {'MCQ': 'What is the estimated age of life on Earth?',
  'Choices': 'a: 1 billion years | b: 3.7 billion years | c: 10 billion years | d: 5 billion years',
  'Correct': 'b'},
 {'MCQ': 'What is the term used to describe the variety of different species in an ecosystem?',
  'Choices': 'a: Biodiversity | b: Evolutionary adaptation | c: Ecological niche | d: Species richness',
  'Corr

In [205]:
quiz=pd.DataFrame(quiz_table_data)

In [206]:
quiz.to_csv("machinelearning.csv",index=False)

In [207]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'11_17_2024_02_10_55'